In [22]:
import pandas as pd
from sklearn import preprocessing


from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.preprocessing import Imputer,StandardScaler,MinMaxScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

In [23]:
df = pd.read_csv('train_examen.csv')
df.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,no checking,6.0,critical/other existing credit,new car,250.0,>=1000,1<=X<4,2.0,female div/dep/mar,none,...,real estate,41.0,bank,own,2.0,unskilled resident,1.0,none,yes,good
1,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
2,0<=X<200,30.0,existing paid,furniture/equipment,3832.0,<100,<1,2.0,male mar/wid,none,...,life insurance,22.0,none,own,1.0,skilled,1.0,none,yes,good
3,0<=X<200,9.0,existing paid,new car,3195.0,no known savings,1<=X<4,1.0,female div/dep/mar,none,...,real estate,33.0,none,own,1.0,unskilled resident,1.0,none,yes,good
4,no checking,9.0,critical/other existing credit,new car,2134.0,<100,1<=X<4,4.0,male single,none,...,car,48.0,none,own,3.0,skilled,1.0,yes,yes,good


In [24]:
X = df.select_dtypes(include=[object])
X.head()

,checking_status,credit_history,purpose,savings_status,employment,personal_status,other_parties,property_magnitude,other_payment_plans,housing,job,own_telephone,foreign_worker,class
0,no checking,critical/other existing credit,new car,>=1000,1<=X<4,female div/dep/mar,none,real estate,bank,own,unskilled resident,none,yes,good
1,<0,critical/other existing credit,radio/tv,no known savings,>=7,male single,none,real estate,none,own,skilled,yes,yes,good
2,0<=X<200,existing paid,furniture/equipment,<100,<1,male mar/wid,none,life insurance,none,own,skilled,none,yes,good
3,0<=X<200,existing paid,new car,no known savings,1<=X<4,female div/dep/mar,none,real estate,none,own,unskilled resident,none,yes,good
4,no checking,critical/other existing credit,new car,<100,1<=X<4,male single,none,car,none,own,skilled,yes,yes,good


In [25]:
le = preprocessing.LabelEncoder()
X_2 = X.apply(le.fit_transform)
X_2.shape

(700, 14)

In [26]:
enc = preprocessing.OneHotEncoder()
enc.fit(X_2)
onehotlabels = enc.transform(X_2).toarray()
onehotlabels.shape

(700, 56)

In [27]:
X = pd.DataFrame(onehotlabels) 
df.replace('good',1,inplace = True)
df.replace('bad',0,inplace = True)
y = df['class']


In [28]:
Xt,Xv,yt,yv =train_test_split(X,y,train_size=0.7)
for d in [Xt,Xv,yt,yv]:
    print d.shape

(489, 56)
(211, 56)
(489,)
(211,)


In [29]:
def metricas(model,Xt,Xv,yt,yv):
    print roc_auc_score(y_score=model.predict_proba(Xt)[:,1],y_true=yt)
    print roc_auc_score(y_score=model.predict_proba(Xv)[:,1],y_true=yv)
    print accuracy_score(y_pred=model.predict(Xt),y_true=yt)
    print accuracy_score(y_pred=model.predict(Xv),y_true=yv)
    print confusion_matrix(y_pred=model.predict(Xt),y_true=yt)
    print confusion_matrix(y_pred=model.predict(Xv),y_true=yv)

In [30]:
model = DecisionTreeClassifier()
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

1.0
1.0
1.0
1.0
[[148   0]
 [  0 341]]
[[ 67   0]
 [  0 144]]


In [31]:
param_grid = dict(criterion=['gini','entropy'],
                 max_depth=range(1,11),
                 min_samples_split = range(100,210,10)
                 )

In [32]:
%%time 
model = DecisionTreeClassifier()
grid = GridSearchCV(cv=10,n_jobs=-1,scoring='roc_auc',
                    estimator=model,
                    param_grid=param_grid)


grid.fit(X,y)

print grid.best_estimator_
print grid.best_params_
print grid.best_score_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=100, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
{'min_samples_split': 100, 'criterion': 'gini', 'max_depth': 1}
1.0
CPU times: user 15.5 s, sys: 532 ms, total: 16 s
Wall time: 51.5 s


In [33]:
model = grid.best_estimator_
model.fit(Xt,yt)
metricas(model,Xt,Xv,yt,yv)

1.0
1.0
1.0
1.0
[[148   0]
 [  0 341]]
[[ 67   0]
 [  0 144]]


In [38]:
Z = pd.read_csv('test_examen.csv')

In [40]:
Z1 = Z.select_dtypes(include=[object])
le = preprocessing.LabelEncoder()
Z = Z1.apply(le.fit_transform)
Z.shape

(300, 13)

In [42]:
enc = preprocessing.OneHotEncoder()
enc.fit(X_2)
onehotlabels = enc.transform(X_2).toarray()
onehotlabels.shape
W = pd.DataFrame(onehotlabels) 

In [45]:
pd.DataFrame(model.predict_proba(W))

,0,1
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0
5,0.0,1.0
6,0.0,1.0
7,0.0,1.0
8,0.0,1.0
9,0.0,1.0
